# Collecting and preparing data

In [1]:
import os, json, requests
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import cbsodata
from datetime import date, timedelta

today = date.today()
base_dir = './data/csvs/'
sources = './data/sources.json'

if os.getcwd().endswith("data"):
    base_dir = './csvs/'
    sources = './sources.json' 

with open(sources, 'r') as json_file:
        resources = json.load(json_file)

def cleanup(df):
    for c in df.columns:
        if 'ersion' in c.lower():
            df.drop(columns=c, inplace=True)
    return df

## (Down)loading data

In [2]:
print('Pulling data from sources.json:')

for org,sets in resources.items():
    for dataset,meta in sets.items():
        name = '_'.join(dataset.lower().split())
        data_loc = base_dir + org.lower() + '_' + name + '.csv'
        updated = meta['updated']
        try:
            os.stat(data_loc)
        except:
            no_csv = True
        else:
            no_csv = False
        if no_csv or updated != str(today):
            if org == 'CBS':
                identifier = meta['code']
                pd.DataFrame(cbsodata.get_data(identifier)).to_csv(data_loc, index=False)
                #metadata = cbsodata.get_info(identifier)
            else:
                if meta['data'][-4:] == '.csv':
                    pd.read_csv(meta['data']).to_csv(data_loc, index=False)
                elif meta['data'][-5:] == '.json':
                    pd.read_json(meta['data']).to_csv(data_loc, index=False)
                else:
                    print('can not handle file format: just dopwnloading file')
                    data_loc = data_loc[-4:]+'.temp'
                    print(data_loc)
                    print(meta['data'])
                    req = requests.get(meta['data'], stream=True)
                    print(req)
                    with open(data_loc[:-5], 'wb') as f:
                        f.write(req.content)  
            meta['updated'] = str(today)
        globals()[name] = pd.read_csv(data_loc)
        print('(Source: '+org+')  '+ name +' @ ' + meta['meta'])

with open(sources, "w") as outfile:
    json.dump(resources, outfile, indent=4)

Pulling data from sources.json:
(Source: CBS)  deceased @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=70895ned&_theme=76
(Source: CBS)  population @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=37230ned&_theme=76
(Source: RIVM)  counts_per_municipality @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427
(Source: RIVM)  prevalence @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/097155aa-75eb-4caa-8ed3-4c6edb80467e
(Source: RIVM)  r @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/ed0699d1-c9d5-4436-8517-27eb993eab6e
(Source: RIVM)  homes_for_elderly @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/0179dd26-7bf6-4021-857f-8623aaf8e73a
(Source: RIVM)  homes_for_disabled @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/aa28b74d-7150-47bc-a7cb-98876e93381b
(Source: RIVM)  seventy_plus_living_at_home @ https://da

## Preparing data

In [3]:
print()
print('Creating dataframes for analysis:')


Creating dataframes for analysis:


### CBS
#### Deceased

In [4]:
# Totals only
if 'ID' in deceased.columns:
    deceased = deceased.drop(columns='ID')
if 'Geslacht' in deceased.columns:
    deceased = deceased.loc[deceased['Geslacht'] == 'Totaal mannen en vrouwen',:]
    deceased.drop(columns='Geslacht', inplace=True)
deceased = deceased.loc[deceased['LeeftijdOp31December'] != 'Totaal leeftijd',:]
deceased.rename(columns={'Overledenen_1':'Deceased'}, inplace=True)

#### Deceased per week

In [5]:
# weeks only, no totals for year
deceased_week_agegroup = deceased[deceased['Perioden'].str.contains('week')]
deceased_week_agegroup['year'] = deceased_week_agegroup['Perioden'].str.slice(stop=4).astype(int)

# split Perioden into year and week
deceased_week_agegroup['week'] = deceased_week_agegroup['Perioden'].str.slice(start=10,stop=12).astype(int)
deceased_week_agegroup = deceased_week_agegroup.loc[deceased_week_agegroup.week != '']
#deceased_week_agegroup['week'] = deceased_week_agegroup['week']

# adding y,week53 to y+1,week00
deceased_week_agegroup.loc[deceased_week_agegroup.loc[:,'week'] == 53, 'year'] += 1
deceased_week_agegroup.loc[deceased_week_agegroup.loc[:,'week'] == 53, 'week'] = 0

# set Index
deceased_week_agegroup.index = pd.MultiIndex.from_frame(deceased_week_agegroup[['year','week']])
deceased_week_agegroup.drop(columns=['year','week', 'Perioden'], inplace=True)
deceased_week_agegroup = deceased_week_agegroup.rename(columns={'Overledenen_1': 'Deceased'})

print('dataframe from cbs created, per week per agegroup: deceased_week_agegroup')
deceased_week_agegroup

dataframe from cbs created, per week per agegroup: deceased_week_agegroup


LeeftijdOp31December  Deceased
year week                               
1995 0           0 tot 65 jaar      62.0
     1           0 tot 65 jaar     476.0
     2           0 tot 65 jaar     511.0
     3           0 tot 65 jaar     473.0
     4           0 tot 65 jaar     513.0
...                        ...       ...
2021 39       80 jaar of ouder    1643.0
     40       80 jaar of ouder    1628.0
     41       80 jaar of ouder    1642.0
     42       80 jaar of ouder    1804.0
     43       80 jaar of ouder    1871.0

[4269 rows x 2 columns]

In [6]:
deceased_per_week = deceased_week_agegroup.groupby(['year','week']).sum()

print('dataframe from cbs created, per week: deceased_per_week')
deceased_per_week

dataframe from cbs created, per week: deceased_per_week


Deceased
year week          
1995 0        394.0
     1       2719.0
     2       2823.0
     3       2609.0
     4       2664.0
...             ...
2021 39      3057.0
     40      3053.0
     41      3034.0
     42      3221.0
     43      3361.0

[1418 rows x 1 columns]

In [7]:
# years only
deceased_year_agegroup = deceased[deceased['Perioden'].str.contains('week') == False]
deceased_year_agegroup = deceased_year_agegroup.rename(columns={'Perioden': 'year'})
deceased_year_agegroup.set_index('year', inplace=True)
deceased_year_agegroup.index = pd.PeriodIndex(deceased_year_agegroup.index, freq='Y')

print('dataframe from cbs created, per year per agegroup: deceased_year_agegroup')
deceased_year_agegroup

dataframe from cbs created, per year per agegroup: deceased_year_agegroup


,LeeftijdOp31December,Deceased
year,,
1995,0 tot 65 jaar,26183.0
1996,0 tot 65 jaar,26388.0
1997,0 tot 65 jaar,25715.0
1998,0 tot 65 jaar,26031.0
1999,0 tot 65 jaar,26557.0
...,...,...
2016,80 jaar of ouder,82349.0
2017,80 jaar of ouder,84095.0
2018,80 jaar of ouder,85820.0


In [8]:
deceased_per_year = deceased_year_agegroup.groupby('year').sum()

print('dataframe from cbs created, per year: deceased_per_year')
deceased_per_year

dataframe from cbs created, per year: deceased_per_year


,Deceased
year,
1995,135675.0
1996,137561.0
1997,135783.0
1998,137482.0
1999,140487.0
2000,140527.0
2001,140377.0
2002,142355.0
2003,141936.0


### Population & Deceased

In [9]:
# dropping unnececary columns
population.drop(columns=['ID','BevolkingAanHetBeginVanDePeriode_1','TotaleVestiging_4',
       'VestigingVanuitEenAndereGemeente_5', 'Immigratie_6',
       'TotaalVertrekInclAdmCorrecties_7', 'VertrekNaarAndereGemeente_8',
       'EmigratieInclusiefAdmCorrecties_9', 'OverigeCorrecties_10',
       'Bevolkingsgroei_11', 'BevolkingsgroeiRelatief_12',
       'BevolkingsgroeiSinds1Januari_13',
       'BevolkingsgroeiSinds1JanuariRela_14'], inplace=True)
population.rename(columns={"LevendGeborenKinderen_2": "Born",
                                        "Overledenen_3": "Deceased",
                                        "BevolkingAanHetEindeVanDePeriode_15": "Population"}, inplace=True)

In [10]:
# RegioS only, no nationals
population = population[population['RegioS'] != 'Nederland']
population = population[population['RegioS'].str.contains('LD') == False]
population = population[population['RegioS'].str.contains('PV') == False]
population = population[population['RegioS'].str.contains('CR') == False]

# Preparing periodes
population['year'] = population.loc[:,'Perioden'].str.slice(stop=4)
population['month'] = population.loc[:,'Perioden'].str.slice(start=5)
population = population.loc[population['month'] != '',:]
population.loc[:,'month'].replace({
    'januari': '01',
    'februari': '02',
    'maart': '03',
    'april': '04',
    'mei': '05',
    'juni': '06',
    'juli': '07',
    'augustus': '08',
    'september': '09',
    'oktober': '10',
    'november': '11',
    'december': '12'}, inplace=True)
population['Y-M'] = population['year'] + '-' + population['month']

population_month_municipality = population.groupby(['Y-M','RegioS']).sum()
population_month_municipality.reset_index(level='RegioS', inplace=True)
population_month_municipality.index = pd.PeriodIndex(population_month_municipality.index, freq='M')

print('dataframe from cbs created, per month per municipality: population_month_municipality')
population_month_municipality

dataframe from cbs created, per month per municipality: population_month_municipality


,RegioS,Born,Deceased,Population
Y-M,,,,
2002-01,'s-Gravendeel,9.0,7.0,8792.0
2002-01,'s-Gravenhage (gemeente),556.0,469.0,458474.0
2002-01,'s-Gravenzande,28.0,16.0,19426.0
2002-01,'s-Hertogenbosch,140.0,98.0,131731.0
2002-01,Aa en Hunze,25.0,21.0,25578.0
...,...,...,...,...
2021-09,Zutphen,32.0,41.0,48477.0
2021-09,Zwartewaterland,34.0,14.0,23009.0
2021-09,Zwijndrecht,51.0,43.0,44812.0


In [11]:
population_year_municipality = population.groupby(['year','RegioS']).sum()
population_year_municipality.reset_index(level='RegioS', inplace=True)
population_year_municipality.index = pd.PeriodIndex(population_year_municipality.index, freq='Y')

print('dataframe from cbs created, per year per municipality: population_year_municipality')
population_year_municipality

dataframe from cbs created, per year per municipality: population_year_municipality


,RegioS,Born,Deceased,Population
year,,,,
2002,'s-Gravendeel,102.0,66.0,106209.0
2002,'s-Gravenhage (gemeente),6328.0,4993.0,5534760.0
2002,'s-Gravenzande,253.0,136.0,233555.0
2002,'s-Hertogenbosch,1728.0,1119.0,1584919.0
2002,Aa en Hunze,289.0,251.0,306187.0
...,...,...,...,...
2021,Zutphen,288.0,349.0,435228.0
2021,Zwartewaterland,216.0,127.0,206126.0
2021,Zwijndrecht,376.0,373.0,402941.0


In [12]:
population_per_month = population.groupby('Y-M').sum()
population_per_month.index = pd.PeriodIndex(population_per_month.index, freq='M')

print('dataframe from cbs created, per month: population_per_month')
population_per_month

dataframe from cbs created, per month: population_per_month


,Born,Deceased,Population
Y-M,,,
2002-01,17019.0,13469.0,16112900.0
2002-02,15448.0,11735.0,16118953.0
2002-03,16792.0,13281.0,16124125.0
2002-04,15995.0,11968.0,16130246.0
2002-05,16800.0,11623.0,16136807.0
...,...,...,...
2021-05,14750.0,13114.0,17500516.0
2021-06,14768.0,12188.0,17507126.0
2021-07,16053.0,12557.0,17515152.0


In [13]:
population_per_year = population.groupby('year').sum()
deceased_per_year = deceased_year_agegroup.groupby('year').sum()
population_per_year = pd.concat([deceased_per_year,population_per_year])
population_per_year.index = pd.PeriodIndex(population_per_year.index, freq='Y')

print('dataframe from cbs created, per month: population_per_year')
population_per_year

dataframe from cbs created, per month: population_per_year


,Deceased,Born,Population
year,,,
1995,135675.0,NaN,NaN
1996,137561.0,NaN,NaN
1997,135783.0,NaN,NaN
1998,137482.0,NaN,NaN
1999,140487.0,NaN,NaN
2000,140527.0,NaN,NaN
2001,140377.0,NaN,NaN
2002,142355.0,NaN,NaN
2003,141936.0,NaN,NaN


### RIVM
#### RIVM per dag

In [14]:
prevalence = cleanup(prevalence)
prevalence = prevalence.groupby('Date').sum()
prevalence.index = pd.PeriodIndex(prevalence.index, freq='D')
rivm_per_day = prevalence

counts_per_municipality.rename(columns={'Date_of_publication':'Date',
                                        'Total_reported': 'Positive_tests',
                                        'Deceased': 'Deceased_covid'
                                       }, inplace=True)
counts_per_municipality = counts_per_municipality.groupby('Date').sum()
counts_per_municipality = counts_per_municipality[['Deceased_covid', 'Positive_tests', 'Hospital_admission']]
counts_per_municipality.index = pd.PeriodIndex(counts_per_municipality.index, freq='D')
rivm_per_day = rivm_per_day.merge(counts_per_municipality, on='Date', how='outer')

hospital_admission.rename(columns={'Date_of_statistics': 'Date',
                                   'Hospital_admission_notification': 'Hospital_admission_NICE_notification',
                                   'Hospital_admission': 'Hospital_admission_NICE'
                                  }, inplace=True)
hospital_admission = cleanup(hospital_admission)
hospital_admission = hospital_admission.groupby('Date').sum()
hospital_admission = hospital_admission[['Hospital_admission_NICE','Hospital_admission_NICE_notification']]
hospital_admission.index = pd.PeriodIndex(hospital_admission.index, freq='D')
rivm_per_day = rivm_per_day.merge(hospital_admission, on='Date', how='outer')

ic_admission.rename(columns={'Date_of_statistics': 'Date'}, inplace=True)
ic_admission = cleanup(ic_admission)
ic_admission = ic_admission.groupby('Date').sum()
ic_admission = ic_admission[['IC_admission','IC_admission_notification']]
ic_admission.index = pd.PeriodIndex(ic_admission.index, freq='D')
rivm_per_day = rivm_per_day.merge(ic_admission, on='Date', how='outer')

hospital_occupation.rename(columns={'Datum': 'Date'}, inplace=True)
hospital_occupation['Date'] = pd.to_datetime(hospital_occupation['Date'], format='%d-%m-%Y')
hospital_occupation['IC_Bedden_TOTAAL'] = hospital_occupation['IC_Bedden_COVID']+hospital_occupation['IC_Bedden_Non_COVID']
hospital_occupation = hospital_occupation.groupby('Date').sum()
hospital_occupation.index = pd.PeriodIndex(hospital_occupation.index, freq='D')
rivm_per_day = rivm_per_day.merge(hospital_occupation, on='Date', how='outer')

r = cleanup(r)
#r.rename(columns={'population': 'source_for_r'}, inplace=True)
r = r.groupby('Date').sum()
#r = r.groupby(['Date','source_for_r']).sum()
#r = r.reset_index(level='source_for_r')
r.index = pd.PeriodIndex(r.index, freq='D')
r = r.loc[r.index <= pd.Period(today-timedelta(days=14), freq='D')]
rivm_per_day = rivm_per_day.merge(r, on='Date', how='outer')

homes_for_elderly.rename(columns={
    'Date_of_statistic_reported': 'Date',
    'Total_cases_reported': 'Positive_tests_elderly_homes',
    'Total_deceased_reported': 'Deceased_elderly_homes',
    'Total_new_infected_locations_reported': 'new_infected_elderly_homes',
    'Total_infected_locations_reported': 'balance_infected_elderly_homes'
    }, inplace=True)
homes_for_elderly = homes_for_elderly.groupby('Date').sum()
homes_for_elderly.index = pd.PeriodIndex(homes_for_elderly.index, freq='D')
homes_for_elderly = homes_for_elderly.loc['2020':]
rivm_per_day = rivm_per_day.merge(homes_for_elderly, on='Date', how='outer')


## Not yet implemented
# homes_for_disabled
# seventy_plus_living_at_home
# sewage
# support_and_behavior
# general_practioner

## STEP FOR ADDING
# rename columns if nececary
# cleanup if nececary
# groupby('Date').sum()
# re-order columns in nececeray
# apply periodIndex(df, freq='D')

#rivm_per_day = rivm_per_day.astype(int)

print('dataframe for RIVM data created, per day: rivm_per_day')
rivm_per_day

dataframe for RIVM data created, per day: rivm_per_day


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Nieuwe_Opnames_COVID,Kliniek_Nieuwe_Opnames_COVID,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-17,693.0,1074.0,1462.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.50,1.93,4.00,NaN,NaN,NaN,NaN
2020-02-18,808.0,1232.0,1653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,1.74,4.00,NaN,NaN,NaN,NaN
2020-02-19,989.0,1461.0,1935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.50,1.59,2.83,NaN,NaN,NaN,NaN
2020-02-20,1188.0,1718.0,2244.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.83,1.74,2.83,NaN,NaN,NaN,NaN
2020-02-21,1466.0,2071.0,2667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.67,2.03,3.67,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-07,NaN,NaN,NaN,8.0,11355.0,54.0,152.0,109.0,34.0,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,131.0,11.0,26.0,508.0
2021-11-08,NaN,NaN,NaN,22.0,11791.0,48.0,186.0,332.0,15.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,223.0,10.0,25.0,526.0
2021-11-09,NaN,NaN,NaN,29.0,11321.0,42.0,142.0,224.0,33.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,158.0,2.0,18.0,540.0


#### RIVM per week

In [15]:
rivm_per_day['year'] = rivm_per_day.index.year
rivm_per_day['week'] = rivm_per_day.index.strftime('%W').astype(int)
rivm_per_week = rivm_per_day.groupby(by=['year', 'week']).sum()

print('dataframe for RIVM data created: rivm_per_week')
rivm_per_week

dataframe for RIVM data created: rivm_per_week


prev_low  prev_avg    prev_up  Deceased_covid  Positive_tests  \
year week                                                                  
2020 7       9143.0   12981.0    16854.0             0.0             0.0   
     8      43236.0   53641.0    63719.0             0.0             5.0   
     9     174623.0  209911.0   240870.0             2.0           158.0   
     10    395330.0  494058.0   582199.0            15.0           946.0   
     11    726956.0  907972.0  1075563.0           162.0          3076.0   
...             ...       ...        ...             ...             ...   
2021 41         0.0       0.0        0.0            44.0         23533.0   
     42         0.0       0.0        0.0            66.0         34841.0   
     43         0.0       0.0        0.0            97.0         49482.0   
     44         0.0       0.0        0.0           150.0         68985.0   
     45         0.0       0.0        0.0           102.0         52152.0   

           Hospital_admission  Hospital_admission_NICE  \
year week                                                
2020 7                    0.0                      0.0   
     8                    2.0                     17.0   
     9                   29.0                     95.0   
     10                 129.0                    434.0   
     11                 816.0                   1690.0   
...                       ...                      ...   
2021 41                 148.0                    467.0   
     42                 254.0                    674.0   
     43                 262.0                    973.0   
     44                 332.0                   1190.0   
     45                 227.0                    379.0   

           Hospital_admission_NICE_notification  IC_admission  \
year week                                                       
2020 7                                      0.0           0.0   
     8                                      0.0           2.0   
     9                                      0.0          18.0   
     10                                     0.0         111.0   
     11                                     0.0         426.0   
...                                         ...           ...   
2021 41                                   399.0          83.0   
     42                                   602.0         118.0   
     43                                   880.0         146.0   
     44                                  1227.0         206.0   
     45                                   746.0          72.0   

           IC_admission_notification  ...  IC_Nieuwe_Opnames_COVID  \
year week                             ...                            
2020 7                           0.0  ...                      0.0   
     8                           0.0  ...                      0.0   
     9                           0.0  ...                      0.0   
     10                          0.0  ...                      0.0   
     11                          0.0  ...                      0.0   
...                              ...  ...                      ...   
2021 41                         74.0  ...                      0.0   
     42                        118.0  ...                      0.0   
     43                        146.0  ...                      0.0   
     44                        194.0  ...                      0.0   
     45                        110.0  ...                      0.0   

           Kliniek_Nieuwe_Opnames_COVID  IC_Bedden_TOTAAL  Rt_low  Rt_avg  \
year week                                                                   
2020 7                              0.0               0.0    3.67   13.38   
     8                              0.0               0.0    9.68   14.39   
     9                              0.0               0.0   10.79   12.90   
     10                             0.0               0.0    8.52    9.43   
     11                             0.0         

In [16]:
#print('dataframe for RIVM data created, per week per municipality: ririvm_week_municipalityvm_per_week')
#rivm_week_municipality

#### RIVM per month

In [17]:
rivm_per_day['Y-M'] = rivm_per_day.index.strftime('%Y') + '-' + rivm_per_day.index.strftime('%m')
rivm_per_month = rivm_per_day.groupby('Y-M').sum()
rivm_per_day.drop(columns='Y-M', inplace=True)
rivm_per_month.index = pd.PeriodIndex(rivm_per_month.index, freq='M')
rivm_per_month = rivm_per_month[rivm_per_month.index < population_per_month.index.max()]

print('dataframe for RIVM data created, per month: rivm_per_month')
rivm_per_month

dataframe for RIVM data created, per month: rivm_per_month


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
Y-M,,,,,,,,,,,,,,,,,,,,,
2020-02,41197.0,53171.0,64978.0,0.0,2.0,1.0,9.0,0.0,1.0,0.0,...,0.0,11.91,25.94,43.07,0.0,0.0,0.0,0.0,26260,97
2020-03,2334676.0,3042902.0,3716756.0,1018.0,12447.0,4636.0,6373.0,4125.0,1546.0,1404.0,...,0.0,31.80,36.69,41.76,1794.0,397.0,582.0,4620.0,62620,328
2020-04,1319824.0,1944858.0,2613234.0,3747.0,26610.0,6056.0,4848.0,6019.0,1113.0,1192.0,...,17913.0,15.87,22.12,28.92,5254.0,1882.0,459.0,22988.0,60600,448
2020-05,347687.0,523427.0,709437.0,1158.0,7137.0,995.0,907.0,1402.0,149.0,162.0,...,26972.0,8.44,24.68,45.25,1279.0,545.0,106.0,13145.0,62620,597
2020-06,81872.0,136389.0,194299.0,163.0,3851.0,154.0,205.0,233.0,47.0,50.0,...,18677.0,12.49,28.12,51.74,169.0,68.0,36.0,2826.0,60600,710
2020-07,114015.0,171938.0,231358.0,59.0,4184.0,83.0,166.0,521.0,31.0,40.0,...,18141.0,34.95,40.57,46.37,58.0,24.0,15.0,720.0,62620,868
2020-08,258720.0,373877.0,490937.0,75.0,16339.0,246.0,468.0,580.0,92.0,97.0,...,19087.0,31.10,33.78,36.48,227.0,51.0,52.0,1047.0,62620,1005
2020-09,1615343.0,2132942.0,2632893.0,187.0,50008.0,577.0,1690.0,1394.0,247.0,255.0,...,19892.0,36.16,37.61,39.07,1022.0,138.0,163.0,2708.0,60600,1104
2020-10,3357599.0,4563484.0,5773497.0,991.0,229558.0,2353.0,7098.0,6818.0,1166.0,1139.0,...,26366.0,30.79,31.50,32.20,5371.0,714.0,584.0,11624.0,62620,1275


In [18]:
#print('dataframe for RIVM data created, per month per municipality: ririvm_month_municipalityvm_per_month')
#rivm_month_municipality

#### RIVM per year

In [19]:
rivm_per_year = rivm_per_day.groupby(rivm_per_day.index.year).sum()
rivm_per_year.index.rename('year', inplace=True)
rivm_per_year.index = pd.PeriodIndex(rivm_per_year.index, freq='Y')
rivm_per_year = rivm_per_year[rivm_per_year.index < population_per_year.index.max()]

print('dataframe for RIVM data created, per month: rivm_per_month')
rivm_per_year

dataframe for RIVM data created, per month: rivm_per_month


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
year,,,,,,,,,,,,,,,,,,,,,
2020,15038960.0,20562299.0,26119189.0,11405.0,795546.0,20036.0,35253.0,34617.0,6580.0,6539.0,...,210074.0,274.17,343.17,428.51,29286.0,6096.0,3048.0,99516.0,644380,9343


In [20]:
#print('dataframe for RIVM data created, per month per municipality: ririvm_month_municipalityvm_per_month')
#rivm_month_municipality

## Merging CBS and RIVM
### Per week deceased_per_week + rivm_per_week

In [21]:
per_week = pd.merge(deceased_per_week, rivm_per_week, left_index=True, right_index=True, how='outer', sort=True)

print('dataframe CBS and RIVM combined, per week: per_week')
per_week

dataframe CBS and RIVM combined, per week: per_week


Deceased  prev_low  prev_avg  prev_up  Deceased_covid  \
year week                                                          
1995 0        394.0       NaN       NaN      NaN             NaN   
     1       2719.0       NaN       NaN      NaN             NaN   
     2       2823.0       NaN       NaN      NaN             NaN   
     3       2609.0       NaN       NaN      NaN             NaN   
     4       2664.0       NaN       NaN      NaN             NaN   
...             ...       ...       ...      ...             ...   
2021 41      3034.0       0.0       0.0      0.0            44.0   
     42      3221.0       0.0       0.0      0.0            66.0   
     43      3361.0       0.0       0.0      0.0            97.0   
     44         NaN       0.0       0.0      0.0           150.0   
     45         NaN       0.0       0.0      0.0           102.0   

           Positive_tests  Hospital_admission  Hospital_admission_NICE  \
year week                                                                
1995 0                NaN                 NaN                      NaN   
     1                NaN                 NaN                      NaN   
     2                NaN                 NaN                      NaN   
     3                NaN                 NaN                      NaN   
     4                NaN                 NaN                      NaN   
...                   ...                 ...                      ...   
2021 41           23533.0               148.0                    467.0   
     42           34841.0               254.0                    674.0   
     43           49482.0               262.0                    973.0   
     44           68985.0               332.0                   1190.0   
     45           52152.0               227.0                    379.0   

           Hospital_admission_NICE_notification  IC_admission  ...  \
year week                                                      ...   
1995 0                                      NaN           NaN  ...   
     1                                      NaN           NaN  ...   
     2                                      NaN           NaN  ...   
     3                                      NaN           NaN  ...   
     4                                      NaN           NaN  ...   
...                                         ...           ...  ...   
2021 41                                   399.0          83.0  ...   
     42                                   602.0         118.0  ...   
     43                                   880.0         146.0  ...   
     44                                  1227.0         206.0  ...   
     45                                   746.0          72.0  ...   

           IC_Nieuwe_Opnames_COVID  Kliniek_Nieuwe_Opnames_COVID  \
year week                                                          
1995 0                         NaN                           NaN   
     1                         NaN                           NaN   
     2                         NaN                           NaN   
     3                         NaN                           NaN   
     4                         NaN                           NaN   
...                            ...                           ...   
2021 41                        0.0                           0.0   
     42                        0.0                           0.0   
     43                        0.0                           0.0   
     44                        0.0                           0.0   
     45                        0.0                           0.0   

           IC_Bedden_TOTAAL  Rt_low  Rt_avg  Rt_up  \
year week                                            
1995 0                  NaN     NaN     NaN    NaN   
     1                  NaN     NaN     NaN    NaN   
     2                  NaN     NaN     NaN    NaN   
     3                  NaN     NaN     NaN    NaN   
     4                  NaN     NaN     NaN

In [22]:
#print('dataframe CBS and RIVM combined, per week per municipality: week_municipality')
#week_municipality

### Per month population_per_month + rivm_per_month

In [23]:
per_month = pd.merge(population_per_month, rivm_per_month, on='Y-M', how='outer', sort=True)

print('dataframe CBS and RIVM combined, per month: per_month')
per_month

#print('dataframe CBS and RIVM combined, per month per municipality: month_municipality')
#month_municipality

dataframe CBS and RIVM combined, per month: per_month


,Born,Deceased,Population,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
Y-M,,,,,,,,,,,,,,,,,,,,,
2002-01,17019.0,13469.0,16112900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02,15448.0,11735.0,16118953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-03,16792.0,13281.0,16124125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-04,15995.0,11968.0,16130246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-05,16800.0,11623.0,16136807.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05,14750.0,13114.0,17500516.0,2127049.0,3050327.0,4013472.0,478.0,153117.0,1733.0,4713.0,...,34702.0,25.59,26.44,27.32,400.0,53.0,107.0,4502.0,62651.0,602.0
2021-06,14768.0,12188.0,17507126.0,488579.0,740897.0,1053081.0,121.0,36135.0,478.0,872.0,...,22481.0,31.25,33.25,35.26,99.0,12.0,28.0,1524.0,60630.0,714.0
2021-07,16053.0,12557.0,17515152.0,10910.0,0.0,77606.0,82.0,180363.0,470.0,1498.0,...,17343.0,34.04,34.96,35.89,591.0,41.0,151.0,1923.0,62651.0,872.0


### Per month population_per_year + rivm_per_year

In [24]:
per_year = pd.merge(population_per_year.drop_duplicates(subset=['Deceased'], keep='last'), rivm_per_year, left_index=True, right_index=True, how='outer', sort=True, copy=False, validate='m:1')
per_year = per_year[(per_year['Born'].isnull() * 1 + (1 - per_year['prev_low'].isnull() * 1)) != 2]

print('dataframe CBS and RIVM combined, per month: per_year')
per_year

#print('dataframe CBS and RIVM combined, per year per municipality: year_municipality')
#year_municipality

dataframe CBS and RIVM combined, per month: per_year


,Deceased,Born,Population,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
year,,,,,,,,,,,,,,,,,,,,,
1995,135675.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,137561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,135783.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,137482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999,140487.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,140527.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,140377.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,142355.0,202083.0,193809414.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,141936.0,200297.0,194705905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## pandas_profiling